In [1]:
# %load_ext autoreload
# %autoreload 1

from IPython.display import display

import numpy as np
import pandas as pd

import os

In [4]:
!cd ..

In [9]:
dataset_path = '/home/aris/Downloads/shapenetcore_partanno_segmentation_benchmark_v0'
result_path = '../results'

baseline_log_path = os.path.join(result_path, 'Log_2021-05-11_06-21-18')
# The "chkp_0300.tar" is unzipped manually
checkpoint_path = os.path.join(baseline_log_path, 'checkpoints', 'chkp_0300.tar')

In [10]:
import torch
from src.models.architectures import KPFCNN
from src.models.train_shapnet_part import ShapeNetPartConfig, ShapeNetPartDataset, ShapeNetPartSampler, DataLoader, ShapeNetPartCollate

NameError: name 'args' is not defined

In [7]:
%tb

SystemExit: 2

In [ ]:
# Initialize configuration class
config = ShapeNetPartConfig()
config.load(baseline_log_path)
config.saving_path = None

# Initialize datasets
training_dataset = ShapeNetPartDataset(config, set='training',
                                       use_potentials=True,
                                       train_category='Airplane')
test_dataset = ShapeNetPartDataset(config, set='validation',
                                   use_potentials=True,
                                   train_category='Airplane')

# Initialize samplers
training_sampler = ShapeNetPartSampler(training_dataset)
test_sampler = ShapeNetPartSampler(test_dataset)

# Initialize the dataloader
training_loader = DataLoader(training_dataset,
                             batch_size=1,
                             sampler=training_sampler,
                             collate_fn=ShapeNetPartCollate,
                             num_workers=config.input_threads,
                             pin_memory=True)
test_loader = DataLoader(test_dataset,
                         batch_size=1,
                         sampler=test_sampler,
                         collate_fn=ShapeNetPartCollate,
                         num_workers=config.input_threads,
                         pin_memory=True)

# Calibrate samplers
training_sampler.calibration(training_loader, verbose=True)
test_sampler.calibration(test_loader, verbose=True)

print('\nModel Preparation')
print('*****************')

# Define network model
net = KPFCNN(config, training_dataset.label_values,
             training_dataset.ignored_labels)